In [432]:
#imports 
import os
import sys
import wget  #keesh added to replace !wget
import argparse  #keesh added to add options

#import caffe  #which version to build?

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as imsave
from IPython import display

import scipy
import scipy.misc

import dicom
from dicom.dataset import Dataset
from dicom.dataset import FileDataset
import datetime
import time
import platform

import natsort
import glob

In [433]:
#constant / setup
IMG_DTYPE = np.float
SEG_DTYPE = np.uint8
plt.set_cmap('gray')

<Figure size 432x288 with 0 Axes>

In [482]:
#functions
""" Image I/O  """
def read_dicom_series(directory, filepattern = "image_*"):
    """ Reads a DICOM Series files in the given directory.
    Only filesnames matching filepattern will be considered"""
    if not os.path.exists(directory) or not os.path.isdir(directory):
        raise ValueError("Given directory does not exist or is a file : " + str(directory))
    print('\tRead Dicom dir: ' + str(directory))
    lstFilesDCM = natsort.natsorted(glob.glob(os.path.join(directory, filepattern)))
    print('\tLength dicom series: ' + str(len(lstFilesDCM)))
    
    # Get ref file
    RefDs = dicom.read_file(lstFilesDCM[0])
    # Load dimensions based on the number of rows, columns, and slices (along the Z axis)
    ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFilesDCM))
    # The array is sized based on 'ConstPixelDims'
    ArrayDicom = np.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)
    Arrayds = [None] * len(lstFilesDCM)
    
    # loop through all the DICOM files
    first_time = True
    wc = ww = 0
    m = 1
    b = 0
    for filenameDCM in lstFilesDCM:
        # read the file
        ds = dicom.read_file(filenameDCM)
        # store the raw image data
        ArrayDicom[:, :, lstFilesDCM.index(filenameDCM)] = ds.pixel_array
        Arrayds[lstFilesDCM.index(filenameDCM)] = ds
        if first_time:
            try:
                wc = int(ds[0x0028, 0x1050].value)   #0028,1050  Window Center: 40
                ww = int(ds[0x0028, 0x1051].value)   #0028,1051  Window Width: 400
                m = float(ds[0x0028, 0x1053].value)  #0028,1053  Rescale Slope: 1 
                b = float(ds[0x0028, 0x1052].value)  #0028,1052  Rescale Intercept: -1024 
                first_time = False
            except:
                wc = ww = 0  #not needed but clearer
                m = 1
                b = 0

    return ArrayDicom, Arrayds, len(lstFilesDCM), wc, ww, m, b

def write_dicom_mask(img_slice, ds_slice, slice_no, outputdirectory, filepattern = ".dcm"):
    (rows, cols) = img_slice.shape
    base_fname = str(slice_no).zfill(6)
    filename = outputdirectory + os.path.sep + base_fname + "_mask1" + filepattern
    
    file_meta = Dataset()
    #will need to generate all UID for real
    file_meta.MediaStorageSOPClassUID = 'Secondary Capture Image Storage'
    file_meta.MediaStorageSOPInstanceUID = '1.3.6.1.4.1.9590.100.1.1.111165684411017669021768385720736873780'
    file_meta.ImplementationClassUID = '1.3.6.1.4.1.9590.100.1.0.100.4.0'
    ds = FileDataset(filename, {}, file_meta = file_meta, preamble="\0"*128)
    ds.Modality = ds_slice.Modality
    ds.ContentDate = str(datetime.date.today()).replace('-','')
    ds.ContentTime = str(time.time()) #milliseconds since the epoch
    
    ds.StudyInstanceUID =  '1.3.6.1.4.1.9590.100.1.1.124313977412360175234271287472804872093'
    ds.SeriesInstanceUID = '1.3.6.1.4.1.9590.100.1.1.369231118011061003403421859172643143649'
    ds.SOPInstanceUID =    '1.3.6.1.4.1.9590.100.1.1.111165684411017669021768385720736873780'
    ds.SOPClassUID = 'Secondary Capture Image Storage'
    ds.SecondaryCaptureDeviceManufacturer = platform.sys.version

    ## These are the necessary imaging components of the FileDataset object.
    ds.SamplesPerPixel = 1
    ds.PhotometricInterpretation = "MONOCHROME2"
    ds.PixelRepresentation = 0
    ds.HighBit = 7
    ds.BitsStored = 8
    ds.BitsAllocated = 8
    ds.SmallestImagePixelValue = '\\x00\\x00'
    ds.LargestImagePixelValue = '\\x01\\x01'
    ds.Columns = cols
    ds.Rows = rows
    ds.PixelData = img_slice.tobytes()
    
    ds.ImplementationVersionName = "pydicom"  #should add version too
    image_type_val = ds_slice[0x0008, 0x0008].value
    image_type_val_str = "\\".join(str(x) for x in image_type_val)
    image_type_val_str2 = image_type_val_str.replace("ORIGINAL", "DERIVED", 1)  
    ds.ImageType = image_type_val_str2
    
    ds.SliceThickness = ds_slice[0x0018, 0x0050].value
    ds.SpacingBetweenSlices = ds_slice[0x0018, 0x0088].value 

    ds.SeriesNumber = ds_slice[0x0020, 0x0011].value
    ds.InstanceNumber = ds_slice[0x0020, 0x0013].value

    ds.ImagePositionPatient = ds_slice[0x0020, 0x0032].value # 0020,0032  Image Position (Patient): 0\0\0
    ds.ImageOrientationPatient = ds_slice[0x0020, 0x0037].value # 0020,0037  Image Orientation (Patient): 1\0\0\0\1\0
    ds.SliceLocation = ds_slice[0x0020, 0x1041].value
    ds.PixelSpacing = ds_slice[0x0028, 0x0030].value # 0028,0030 Pixel Spacing 0.742999970912933\0.742999970912933  
    
    #Display components
    ds.WindowCenter = [0]   #0028,1050  Window Center
    ds.WindowWidth = [0]  #0028,1051  Window Width
    ds.RescaleIntercept = 0  #0028,1052  Rescale Intercept: 0
    ds.RescaleSlope = 1 #0028,1053  Rescale Slope: 1 
      
    ds.save_as(filename)

    
""" Image Preprocessing """
def to_scale(img, shape=None):
    height, width = shape
    if img.dtype == SEG_DTYPE:
        # This function is only available if Python Imaging Library (PIL) is installed.
        # 888 Interpolation to use for re-sizing ('nearest', 'lanczos', 'bilinear', 'bicubic' or 'cubic').
        return scipy.misc.imresize(img,(height,width),interp="nearest").astype(SEG_DTYPE)
    elif img.dtype == IMG_DTYPE:
        max_ = np.max(img)
        factor = 255.0/max_ if max_ != 0 else 1
        return (scipy.misc.imresize(img,(height,width),interp="nearest")/factor).astype(IMG_DTYPE)
    else:
        raise TypeError('Error. To scale the image array, its type must be np.uint8 or np.float64. (' + str(img.dtype) + ')')


def normalize_image(img):
    """ Normalize image values to [0,1] """
    min_, max_ = float(np.min(img)), float(np.max(img))
    return (img - min_) / (max_ - min_)

def normalize_image_using_rescale_slope_intercept(img, m, b):
    """ Normalize image values to y = mx + b """
    return ((m * img) + b)

def byte_normalize_image(img):
    """ Normalize image values to [0,1] """
    min_, max_ = float(np.min(img)), float(np.max(img))
    return (255.0 * (img - min_) / (max_ - min_))


def histeq_processor(img):
    """Histogram equalization"""
    nbr_bins=256
    #get image histogram
    imhist,bins = np.histogram(img.flatten(),nbr_bins,normed=True)
    cdf = imhist.cumsum() #cumulative distribution function
    cdf = 255 * cdf / cdf[-1] #normalize
    #use linear interpolation of cdf to find new pixel values
    original_shape = img.shape
    img = np.interp(img.flatten(),bins[:-1],cdf)
    img=img/255.0
    return img.reshape(original_shape)

In [483]:
#disaply function
""" Image Stats / Display"""
def stat(array):
    #may need str casts?
    print('min: ' + np.min(array) + ' max: ' + np.max(array) + ' median: ' + np.median(array) + ' avg: ' + np.mean(array))


def imshow(*args,**kwargs):
    """ Handy function to show multiple plots in on row, possibly with different cmaps and titles
    Usage:
    imshow(img1, title="myPlot")
    imshow(img1,img2, title=['title1','title2'])
    imshow(img1,img2, cmap='hot')
    imshow(img1,img2,cmap=['gray','Blues']) """
    cmap = kwargs.get('cmap', 'gray')
    title= kwargs.get('title','')
    if len(args) == 0:
        raise ValueError("No images given to imshow")
    elif len(args) == 1:
        plt.title(title)
        plt.imshow(args[0], interpolation='none')
    else:
        n = len(args)
        if type(cmap) == str:
            cmap = [cmap]*n
        if type(title) == str:
            title= [title]*n
        plt.figure(figsize=(n*5,10))
        for i in range(n):
            plt.subplot(1,n,i+1)
            plt.title(title[i])
            plt.imshow(args[i], cmap[i])
    plt.show()

In [484]:
def step1_preprocess_img_slice(img_slc, slice, wc, ww, m, b, results_dir):
    """
    Preprocesses the image 3d volumes by performing the following :
    1- Rotate the input volume so the the liver is on the left, spine is at the bottom of the image
    2- Set pixels with hounsfield value great than 1200, to zero.
    3- Clip all hounsfield values to the range [-100, 400]
    4- Normalize values to [0, 1]
    5- Rescale img and label slices to 388x388
    6- Pad img slices with 92 pixels on all sides (so total shape is 572x572)

    Args:
        img_slc: raw image slice
    Return:
        Preprocessed image slice
    """
    img_slc   = img_slc.astype(IMG_DTYPE)
    
    #apply m and b
    img_slc = normalize_image_using_rescale_slope_intercept(img_slc, m, b)
    
    img_dbl = img_slc
    img_slc[img_slc>1200] = 0

    thresh_lo = -100
    thresh_hi = 400
    if wc != 0 and ww != 0:
        thresh_lo = wc - (ww / 2)
        thresh_hi = wc + (ww / 2)
    print("HU thresh low= " + str(thresh_lo))
    print("HU thresh high= " + str(thresh_hi))
    
    print("m= " + str(m))
    print("b= " + str(b))

    img_slc = np.clip(img_slc, thresh_lo, thresh_hi)
    img_hu = img_slc

    # imshow(img_dbl, img_hu, img_slc, ["double", "HU thresholded" , "final pp"])
    
    img_slc   = normalize_image(img_slc)
    img_slc   = to_scale(img_slc, (388,388))
    img_slc   = np.pad(img_slc,((92,92),(92,92)),mode='reflect')

    # keesh -- this was not executed in original notebook and seemd to make results worse!
    #img_slc = histeq_processor(img_slc)
    
    return img_slc

In [485]:
os.getcwd()

'C:\\Users\\eric'

In [486]:
# action block
dir = "E:\\data\\CDor_3\\test1\\05-09-2006-CT ABDOMEN NONENH  ENHANCED-BODY-10426\\3-TRI LIVER C--33804"

In [487]:
img, ds, num_images, wc, ww, m, b = read_dicom_series(dir + os.path.sep, filepattern="*.dcm")

slice_no = 90  # 0-based and is in order 
img_slc = img[..., slice_no]
ds_slc = ds[slice_no]

img_slc = img_slc.astype(IMG_DTYPE)
img_slc = normalize_image_using_rescale_slope_intercept(img_slc, m, b)
img_slc = byte_normalize_image(img_slc)

img_slc = to_scale(img_slc, (400,400))
img_slc = img_slc.astype(SEG_DTYPE)

write_dicom_mask(img_slc, ds_slc, slice_no, "E:\\data\\CDor_3\\test_write_dcm", filepattern = ".dcm")

	Read Dicom dir: E:\data\CDor_3\test1\05-09-2006-CT ABDOMEN NONENH  ENHANCED-BODY-10426\3-TRI LIVER C--33804\
	Length dicom series: 113


D:\ProgramData\Anaconda3\envs\fcfn_preproc\lib\site-packages\ipykernel_launcher.py:115: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


In [440]:
#img_p = step1_preprocess_img_slice(img[...,100], 100, wc, ww, m, b, "")
#imshow(img_p, "Preproced image")